In [6]:
import numpy as np 
import pandas as pd 
import pandas_datareader as web
#import pandas.io.data as web 
from pandas_datareader import data, wb
import jqdatasdk as jd
import pandas_datareader.data as web


In [ ]:
sp500= data.DataReader('^GSPC', data_source='yahoo', start='1/1/2000', end='1/12/2017') 
#sp500= data.DataReader('^GSPC', data_source='yahoo') 
sp500.ix['2010-01-04'] 
sp500.info() 
print(sp500) 
print(sp500.columns) 
print(sp500.shape)
import matplotlib.pyplot as plt 
plt.plot(sp500['Close'])
# now calculating the 42nd Days and 252 days trend for the index 
sp500['42d']= np.round(pd.rolling().mean(sp500['Close'], window=42),2) 
sp500['252d']= np.round(pd.rolling().mean(sp500['Close'], window=252),2)
#Look at the data 
sp500[['Close','42d','252d']].tail() 
plt.plot(sp500[['Close','42d','252d']])

In [ ]:
import cherrypy

class HelloWorld(object):
    @cherrypy.expose
    def index(self):
        return "Hello World!"

cherrypy.quickstart(HelloWorld())

[25/Jun/2019:11:24:52] ENGINE Listening for SIGTERM.
[25/Jun/2019:11:24:52] ENGINE Listening for SIGHUP.
[25/Jun/2019:11:24:52] ENGINE Listening for SIGUSR1.
[25/Jun/2019:11:24:52] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[25/Jun/2019:11:24:52] ENGINE Started monitor thread 'Autoreloader'.
[25/Jun/2019:11:24:52] ENGINE Serving on http://127.0.0.1:8080
[25/Jun/2019:11:24:52] ENGINE Bus STARTED


In [8]:
from hmmlearn.hmm import GaussianHMM
import numpy as np
from matplotlib import cm, pyplot as plt
import matplotlib.dates as dates
import pandas as pd
import datetime

In [15]:
beginDate = '2005-01-01'
endDate = '2015-12-31'
n = 6 #6个隐藏状态
#data = get_price('CSI300.INDX',start_date=beginDate, end_date=endDate,frequency='1d')
#df = rd.get_price('CSI300.INDX', '2005-01-01', '2015-07-25').reset_index()[['OpeningPx', 'ClosingPx']]
data=jd.get_index_stocks('000300.XSHG',start_date=beginDate, end_date=endDate,frequency='1d')
data = web.DataReader("AMZN","quandl",
                      datetime.datetime(2005,1,1),
                      datetime.datetime(2015,1,1),
                      access_key = 'RGqizhsWNp9P1gM7wx5u'
                     ).sort_index()
data[0:9]

run jqdatasdk.auth first


,Open,High,Low,Close,Volume,ExDividend,SplitRatio,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjVolume
Date,,,,,,,,,,,,
2005-01-03,44.95,45.44,44.21,44.52,10446500.0,0.0,1.0,44.95,45.44,44.21,44.52,10446500.0
2005-01-04,42.62,43.26,41.50,42.14,19418500.0,0.0,1.0,42.62,43.26,41.50,42.14,19418500.0
2005-01-05,41.57,42.76,41.56,41.77,8354200.0,0.0,1.0,41.57,42.76,41.56,41.77,8354200.0
2005-01-06,41.81,42.25,40.90,41.05,8700900.0,0.0,1.0,41.81,42.25,40.90,41.05,8700900.0
2005-01-07,41.33,42.69,41.16,42.32,9836600.0,0.0,1.0,41.33,42.69,41.16,42.32,9836600.0
2005-01-10,42.05,42.96,41.71,41.84,7347900.0,0.0,1.0,42.05,42.96,41.71,41.84,7347900.0
2005-01-11,41.40,42.16,41.01,41.64,7920300.0,0.0,1.0,41.40,42.16,41.01,41.64,7920300.0
2005-01-12,41.49,42.48,40.82,42.30,8072300.0,0.0,1.0,41.49,42.48,40.82,42.30,8072300.0
2005-01-13,42.45,44.58,42.34,42.60,17393600.0,0.0,1.0,42.45,44.58,42.34,42.60,17393600.0


In [18]:
#拿到每日成交量和收盘价的数据。
volume = data['Volume']
close = data['Close']

In [20]:
#计算每日最高最低价格的对数差值，作为特征状态的一个指标。
logDel = np.log(np.array(data['High'])) - np.log(np.array(data['Low']))
logDel

array([0.02744177, 0.04153499, 0.02846492, ..., 0.02477376, 0.01476089,
       0.00953474])

In [21]:
#计算每5日的指数对数收益差，作为特征状态的一个指标。
logRet_1 = np.array(np.diff(np.log(close)))#这个作为后面计算收益使用
logRet_5 = np.log(np.array(close[5:])) - np.log(np.array(close[:-5]))
logRet_5

array([-0.06208571, -0.01193616,  0.01260871, ...,  0.0396823 ,
        0.01219132,  0.01318465])

In [23]:
#计算每5日的指数成交量的对数差，作为特征状态的一个指标。
logVol_5 = np.log(np.array(volume[5:])) - np.log(np.array(volume[:-5]))
logVol_5

array([-0.35185244, -0.89679714, -0.03432596, ..., -1.0627487 ,
       -0.64864122, -0.27840746])

In [24]:
#由于计算中出现了以5天为单位的计算，所以要调整特征指标的长度。
logDel = logDel[5:]
logRet_1 = logRet_1[4:]
close = close[5:]
Date = pd.to_datetime(data.index[5:])

In [33]:
#把我们的特征状态合并在一起。
A = np.column_stack([logDel,logRet_5,logVol_5])
A

array([[ 0.02952854, -0.06208571, -0.35185244],
       [ 0.02765596, -0.01193616, -0.89679714],
       [ 0.03986122,  0.01260871, -0.03432596],
       ...,
       [ 0.02477376,  0.0396823 , -1.0627487 ],
       [ 0.01476089,  0.01219132, -0.64864122],
       [ 0.00953474,  0.01318465, -0.27840746]])

In [39]:
#model = GaussianHMM(n_components= n, covariance_type="full", n_iter=2000).fit([A])
model = GaussianHMM(n_components= n, covariance_type="full", n_iter=2000).fit([A])
hidden_states = model.predict(A)
hidden_states

ValueError: Found array with dim 3. Estimator expected <= 2.

 help(GaussianHMM)

In [2]:
import copy
import datetime
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates
from matplotlib.dates import (
DateFormatter,WeekdayLocator,DayLocator,MONDAY
)
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from sklearn.cluster import KMeans



start = datetime.datetime(2013,1,1)
end = datetime.datetime(2015,12,31)
sp500 = web.DataReader('CHRIS/CME_SP1','quandl',start,end,access_key = 'RGqizhsWNp9P1gM7wx5u')

#plot_candlesticks(sp500,datetime.datetime(2014,6,1))
data = sp500
since = datetime.datetime(2014,6,1)
df = copy.deepcopy(data)
df = df[df.index >= since]
df.reset_index(inplace = True)

df
# mondays = WeekdayLocator(MONDAY)
# alldays = DayLocator()
# weekFormatter = DateFormatter('%b %d')
# fig,ax = plt.subplots(figsize = (16,4))
# fig.subplots_adjust(bottom = 0.2)
# ax.xaxis.set_major_locator(mondays)
# ax.xaxis.set_minor_locator(alldays)
# ax.xaxis.set_major_formatter(weekFormatter)

# csticks = candlestick_ohlc(ax,df[['date_fmt','Open','High','Low','Last']].values,width = 0.6,
#                            colorup = '#000000',colordown = '#ff0000')
# ax.set_facecolor((1,1,0.9))  
# plt.setp(plt.gca().get_xticklabels(),
#         rotation = 45,horizontalalignment = 'right')
# plt.show()

# sp500_norm = get_open_normalized_prices(sp500)
# k = 5
# km = KMeans(n_clusters = k,random_state = 42)
# km.fit(sp500_norm)
# labels = km.labels_
# sp500['Cluster'] = labels

# #plot_3d_normalized_candles(sp500_norm)
# data = sp500_norm
# fig = plt.figure(figsize = (12,9))
# ax = Axes3D(fig,elev = 21,azim = -136)
# ax.scatter(
#     data['H/O'],data['L/O'],data['C/O'],c = labels.astype(np.float))
# ax.set_xlabel('High/Open')
# ax.set_ylabel('Low/Open')
# ax.set_zlabel('Close/Open')
# plt.show()

,Date,Open,High,Low,Last,Change,Settle,Volume,PreviousDayOpenInterest
0,2015-12-31,2054.5,2057.1,2030.5,2036.5,19.2,2035.4,4211.0,80832.0
1,2015-12-30,2072.3,2074.2,2054.0,2054.5,18.2,2054.6,3828.0,79487.0
2,2015-12-29,2049.7,2074.0,2048.0,2072.0,24.0,2072.8,8080.0,78894.0
3,2015-12-28,2052.4,2053.4,2035.5,2050.5,2.4,2048.8,4504.0,76472.0
4,2015-12-24,2052.9,2059.3,2045.0,2045.0,1.8,2051.2,652.0,79680.0
5,2015-12-23,2034.2,2057.8,2031.5,2053.0,17.1,2053.0,7422.0,80015.0
6,2015-12-22,2020.6,2036.3,2007.8,2036.1,21.0,2035.9,3563.0,79568.0
7,2015-12-21,1999.0,2019.0,1995.9,2015.0,23.0,2014.9,1227.0,79639.0
8,2015-12-18,2024.0,2028.9,1991.5,1993.0,32.8,1991.9,12732.0,70338.0
9,2015-12-17,2078.0,2078.0,2037.0,2037.0,37.8,2034.3,9275.0,62425.0
